In [5]:
!pip install karateclub

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install keras-embed-sim

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [1]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.layers import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras_preprocessing.sequence import pad_sequences
import keras

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import amrlib
stog = amrlib.load_stog_model()

In [27]:
df=pd.read_csv(r'all_true_fake_merged.csv')

In [32]:
df['tweet']=df['tweet'].str.replace(r'http\S+', '').str.replace(r'@\S+','').str.replace(r'#\S+','').str.replace(r'|','').str.replace(r',','').str.replace(r'-','').str.replace(r'\'','').str.replace(r'%','').str.replace(r'\d','').str.replace(r'\n','')#.str.replace(r';','.')

/DATA/gupta37/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/DATA/gupta37/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [33]:
df['no_of_sent']=0

In [34]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
df['no_of_sent']= df['tweet'].apply(lambda x: len(sent_tokenize(x)))
# for i in range(df.shape[0]):
#     df['no_of_sent'][i] = len(sent_tokenize(df['tweet'][i]))

[nltk_data] Downloading package punkt to /DATA/gupta37/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
df[df['no_of_sent']>1][:6]

,author id,created_at,mentions_id,mentions_username,hashtags,geo,longitude,latitude,id,lang,like_count,quote_count,reply_count,retweet_count,follower_count,source,tweet,target,no_of_sent
20,1.039760e+18,2022-06-07 10:09:34+00:00,[],[],"['NupurSharma', 'BJP', 'PMModi']",NaN,NaN,NaN,1.534115e+18,hi,11.0,0.0,0.0,0.0,218235.0,TweetDeck,After all what happened that BJP took such a b...,1,2
25,1.039760e+18,2022-06-07 05:04:00+00:00,[],[],"['NupurSharmaCase', 'IndiaAndGulfRelation']",NaN,NaN,NaN,1.534038e+18,hi,5.0,0.0,0.0,1.0,218235.0,TweetDeck,Nupur Sharma is protesting in the case how man...,1,2
75,1.039760e+18,2022-06-09 16:16:38+00:00,[],[],"['TV9D', 'ControversialRemarksAgainstProphetMu...",NaN,NaN,NaN,1.534932e+18,und,2.0,0.0,0.0,1.0,218233.0,TweetDeck,Understand the difference between Nupur Sharma...,1,2
80,1.039760e+18,2022-06-11 10:05:26+00:00,"['324467993', '209663295']","['Surbhi_R_Sharma', 'GauravAgrawaal']","['NupurSharmaControversy', 'JamaMasjid', 'UPAl...",NaN,NaN,NaN,1.535564e+18,hi,13.0,0.0,1.0,4.0,218233.0,Twitter Media Studio - LiveCut,Inflation reached . percent in US in May ...,1,2
90,1.039760e+18,2022-06-11 07:33:51+00:00,['1502627969722105867'],['DivyaKocharr'],"['TV9Digital', 'nupursharma', 'prophetmuhammad']",NaN,NaN,NaN,1.535526e+18,und,4.0,0.0,2.0,1.0,218233.0,Twitter Media Studio,Why did Qatar first raise anti India flag in N...,1,2
173,1.039760e+18,2022-06-13 11:36:21+00:00,[],[],"['Kuwait', 'NupurSharma']",NaN,NaN,NaN,1.536312e+18,hi,10.0,0.0,0.0,2.0,218235.0,TweetDeck,Those who performed in Kuwait will have to ret...,1,2


In [36]:
df[df['no_of_sent']==0]

,author id,created_at,mentions_id,mentions_username,hashtags,geo,longitude,latitude,id,lang,like_count,quote_count,reply_count,retweet_count,follower_count,source,tweet,target,no_of_sent
32,1.039760e+18,2022-06-07 15:08:01+00:00,[],[],"['News', 'LatestNews', 'ProphetMuhammad']",NaN,NaN,NaN,1.534190e+18,hi,9.0,0.0,3.0,0.0,218233.0,TweetDeck,,1,0
40,1.039760e+18,2022-06-09 04:50:20+00:00,[],[],"['ProphetMuhammad', 'NupurSharma', 'AMuS']",NaN,NaN,NaN,1.534760e+18,hi,9.0,1.0,7.0,0.0,218235.0,TweetDeck,,1,0
42,1.039760e+18,2022-06-10 13:21:56+00:00,"['250246659', '3305343308', '90098179537345740...","['ProfKapilKumar', 'QasmiMufti', 'avanijeshDU'...","['Adi', 'NupurSharmaControversy', 'JamaMasjid'...",NaN,NaN,NaN,1.535251e+18,hi,37.0,2.0,6.0,11.0,218233.0,Twitter Media Studio - LiveCut,,1,0
43,1.039760e+18,2022-06-10 13:19:16+00:00,"['250246659', '3305343308', '90098179537345740...","['ProfKapilKumar', 'QasmiMufti', 'avanijeshDU'...","['Adi', 'NupurSharmaControversy', 'JamaMasjid'...",NaN,NaN,NaN,1.535250e+18,hi,15.0,0.0,1.0,5.0,218233.0,Twitter Media Studio - LiveCut,,1,0
47,1.039760e+18,2022-06-10 13:14:16+00:00,"['250246659', '3305343308', '90098179537345740...","['ProfKapilKumar', 'QasmiMufti', 'avanijeshDU'...","['Adi', 'NupurSharmaControversy', 'JamaMasjid'...",NaN,NaN,NaN,1.535249e+18,hi,10.0,0.0,1.0,3.0,218233.0,Twitter Media Studio - LiveCut,,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8971,1.338488e+18,2022-07-02 06:11:52+00:00,[],[],[],NaN,NaN,NaN,1.543115e+18,qme,0.0,0.0,0.0,0.0,8788.0,Twitter Web App,,0,0
8974,1.338488e+18,2022-07-07 12:32:22+00:00,"['1469675106', '1207634475254865921', '1034742...","['BPPIND', 'PIBFactCheck', 'timesfactcheck', '...",[],NaN,NaN,NaN,1.545023e+18,qme,1.0,1.0,0.0,1.0,8788.0,Twitter Web App,,0,0
9136,1.338488e+18,2022-07-02 06:16:57+00:00,['1503977335329988608'],['beer_shishir'],[],NaN,NaN,NaN,1.543116e+18,qme,0.0,0.0,0.0,0.0,8788.0,Twitter Web App,,0,0
9138,1.338488e+18,2022-07-02 06:11:52+00:00,[],[],[],NaN,NaN,NaN,1.543115e+18,qme,0.0,0.0,0.0,0.0,8788.0,Twitter Web App,,0,0


In [37]:
import networkx as nx

__all__ = ["convert_node_labels_to_integers", "relabel_nodes"]

def relabel_nodes(G, mapping, copy=True):
    m = {n: mapping(n) for n in G} if callable(mapping) else mapping

    if copy:
        return _relabel_copy(G, m)
    else:
        return _relabel_inplace(G, m)



def _relabel_inplace(G, mapping):
    if len(mapping.keys() & mapping.values()) > 0:
        # labels sets overlap
        # can we topological sort and still do the relabeling?
        D = nx.DiGraph(list(mapping.items()))
        D.remove_edges_from(nx.selfloop_edges(D))
        try:
            nodes = reversed(list(nx.topological_sort(D)))
        except nx.NetworkXUnfeasible as err:
            raise nx.NetworkXUnfeasible(
            ) from err
    else:
        # non-overlapping label sets, sort them in the order of G nodes
        nodes = [n for n in G if n in mapping]

    multigraph = G.is_multigraph()
    directed = G.is_directed()

    for old in nodes:
        # Test that old is in both mapping and G, otherwise ignore.
        try:
            new = mapping[old]
            G.add_node(new, **G.nodes[old])
        except KeyError:
            continue
        if new == old:
            continue
        if multigraph:
            new_edges = [
                (new, new if old == target else target, key, data)
                for (_, target, key, data) in G.edges(old, data=True, keys=True)
            ]
            if directed:
                new_edges += [
                    (new if old == source else source, new, key, data)
                    for (source, _, key, data) in G.in_edges(old, data=True, keys=True)
                ]
            # Ensure new edges won't overwrite existing ones
            seen = set()
            for i, (source, target, key, data) in enumerate(new_edges):
                if target in G[source] and key in G[source][target]:
                    new_key = 0 if not isinstance(key, (int, float)) else key
                    while new_key in G[source][target] or (target, new_key) in seen:
                        new_key += 1
                    new_edges[i] = (source, target, new_key, data)
                    seen.add((target, new_key))
        else:
            new_edges = [
                (new, new if old == target else target, data)
                for (_, target, data) in G.edges(old, data=True)
            ]
            if directed:
                new_edges += [
                    (new if old == source else source, new, data)
                    for (source, _, data) in G.in_edges(old, data=True)
                ]
        G.remove_node(old)
        G.add_edges_from(new_edges)
    return G


def _relabel_copy(G, mapping):
    H = G.__class__()
    H.add_nodes_from(mapping.get(n, n) for n in G)
    H._node.update((mapping.get(n, n), d.copy()) for n, d in G.nodes.items())
    if G.is_multigraph():
        new_edges = [
            (mapping.get(n1, n1), mapping.get(n2, n2), k, d.copy())
            for (n1, n2, k, d) in G.edges(keys=True, data=True)
        ]

        # check for conflicting edge-keys
        undirected = not G.is_directed()
        seen_edges = set()
        for i, (source, target, key, data) in enumerate(new_edges):
            while (source, target, key) in seen_edges:
                if not isinstance(key, (int, float)):
                    key = 0
                key += 1
            seen_edges.add((source, target, key))
            if undirected:
                seen_edges.add((target, source, key))
            new_edges[i] = (source, target, key, data)

        H.add_edges_from(new_edges)
    else:
        H.add_edges_from(
            (mapping.get(n1, n1), mapping.get(n2, n2), d.copy())
            for (n1, n2, d) in G.edges(data=True)
        )
    H.graph.update(G.graph)
    return H


def convert_node_labels_to_integers(
    G, first_label=0, ordering="default", label_attribute=None
):
    N = G.number_of_nodes() + first_label
    if ordering == "default":
        mapping = dict(zip(G.nodes(), range(first_label, N)))
    elif ordering == "sorted":
        nlist = sorted(G.nodes())
        mapping = dict(zip(nlist, range(first_label, N)))
    elif ordering == "increasing degree":
        dv_pairs = [(d, n) for (n, d) in G.degree()]
        dv_pairs.sort()  # in-place sort from lowest to highest degree
        mapping = dict(zip([n for d, n in dv_pairs], range(first_label, N)))
    elif ordering == "decreasing degree":
        dv_pairs = [(d, n) for (n, d) in G.degree()]
        dv_pairs.sort()  # in-place sort from lowest to highest degree
        dv_pairs.reverse()
        mapping = dict(zip([n for d, n in dv_pairs], range(first_label, N)))
    else:
        raise nx.NetworkXError(f"Unknown node ordering: {ordering}")
    H = relabel_nodes(G, mapping)
    # create node attribute with the old label
    if label_attribute is not None:
        nx.set_node_attributes(H, {v: k for k, v in mapping.items()}, label_attribute)
    return H


In [14]:
import networkx as nx
import penman
from karateclub import Graph2Vec
from karateclub import GL2Vec
from karateclub import Node2Vec

In [41]:
sent=[]
for i in range(10):
    for j in sent_tokenize(df['tweet'][i]):
        sent.append(j)

In [42]:
sent

['Delhi: The dome of the Jama Masjid damaged after thunderstorms and heavy rains damage to the tower.',
 'Death of a person',
 'The most elevated symbol of the Mughalia era is Jama Masjid where Maulana Azad and Swami Shraddhanand gave the argument',
 'What is the story behind Nupur Sharmas suspension from BJP and Naveen Jindals dismissal?',
 'After BJPs action Nupur Sharma expressed regret over his statement said could not tolerate insult to Mahadev',
 'Who are BJP spokesperson Nupur Sharma and Naveen Jindal who suspended the party after commenting on Prophet Mohammed?',
 'BJP spokesperson Nupur Sharma and Naveen Jindal suspended for commenting on Prophet Mohammed',
 'Agnipath Scheme: Information about Agneepath scheme given to PM Modi top military officials told how the scheme will work',
 'Why do the Arab countries who flares up on Nupur Sharmas statement do not peek in their own Giriban?',
 'Prophet Mohammad comment dispute Pakistans entry in charge of Indian High Commission summone

In [43]:
graphs=stog.parse_sents(sent)

In [75]:
import penman
import networkx as nx

# Define a function to extract nodes and edges from an AMR graph
def extract_graph(amr):
    nodes = {}
    edges = []
    for triple in amr.triples:
        if triple[1] == ':instance':
            nodes[triple[2]] = triple[0]
        elif triple[1].startswith(':'):
            edges.append((triple[0], triple[1][1:], triple[2]))
        elif triple[1] == 'TOP':
            pass
        else:
            nodes[triple[0]][triple[1]] = triple[2]
    return nodes, edges

# Define a function to merge multiple AMR graphs into a single knowledge graph
def merge_graphs(graphs):
    merged_nodes = {}
    merged_edges = []
    for nodes, edges in graphs:
        for node_id, node_data in nodes.items():
            if node_id not in merged_nodes:
                merged_nodes[node_id] = node_data
#                 merged_nodes[node_id] = node_data
#             else:
#                 merged_nodes[node_id].update(node_data)
        merged_edges.extend(edges)
    return merged_nodes, merged_edges



In [76]:
single_graph_lst=[]
for graph in graphs:
    amr = penman.decode(graph)
    nodes, edges = extract_graph(amr)
    single_graph_lst.append((nodes,edges))

In [77]:
merged_nodes, merged_edges = merge_graphs(single_graph_lst)

In [2]:
# Create a NetworkX graph from the merged nodes and edges
graph = nx.Graph()
for node_id, node_data in merged_nodes.items():
    graph.add_node(node_id, **node_data)
for source, relation, target in merged_edges:
    graph.add_edge(source, target, label=relation)

In [16]:
graphs_list=[]

In [17]:
print(len(df))

9309


In [18]:
# for i in range(100):
#     print(i)
#     sents = sent_tokenize(df['tweet'][i])
#     if len(sents)>0:
#         for j  in  range(len(sents)):
#             #print(len(graphs))
#             graphs=stog.parse_sents([sents[j]])
#             gph = nx.Graph()
#             #print(graphs[j])
#             g = penman.decode(graphs[0])
#             if len(g.edges())>0:
#                 for i in range(len(g.edges())):
#                     gph.add_edges_from([(g.edges()[i][0],g.edges()[i][2])])
#                 gph=convert_node_labels_to_integers(gph, first_label=0, ordering='default', label_attribute=None)
#                 graphs_list.append(gph)
#                 #print(graphs_list)
#             else:
#                 print('list is empty')
#                 graphs_list.append(gph)
                
#     else:
#         gph = nx.Graph()
#         print('list is empty')
#         graphs_list.append(gph)



In [19]:

# with open('fake_amr_graph_lst.pkl', 'wb') as f:
#        pickle.dump(graphs_list, f)

In [18]:
with open('fake_amr_graph_lst.pkl', 'rb') as f:
    graphs_list = pickle.load(f)

In [26]:
len(g2ve)

11618

In [25]:
model = Graph2Vec(dimensions=200)
model.fit(graphs_list)
g2ve=model.get_embedding()

In [129]:
model = GL2Vec(dimensions=500)
model.fit(graphs_list)
gl2ve=model.get_embedding()

In [152]:
graphs_list[0].number_of_nodes()

16

In [153]:
model = Node2Vec()
model.fit(graphs_list[0])
n2ve=model.get_embedding()

In [156]:
graphs_list[0].number_of_nodes()

16

In [38]:
for i in (df[df['no_of_sent']>1].index):
    g2ve[i]=np.mean(g2ve[i:i+df['no_of_sent'][i]],axis=0)
    g2ve=np.delete(g2ve,np.arange(i+1,i+df['no_of_sent'][i]),axis=0)
    

In [132]:
for i in (df[df['no_of_sent']>1].index):
    gl2ve[i]=np.mean(gl2ve[i:i+df['no_of_sent'][i]],axis=0)
    gl2ve=np.delete(gl2ve,np.arange(i+1,i+df['no_of_sent'][i]),axis=0)
    

In [88]:
len(gl2ve)

9309

In [47]:
#tweet=['His grandmother looked after him. He was looked after by his grandmother.','They will send him away to school.He will be sent away to school.']

# TFIDF

In [146]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
sklearn_tfidf = TfidfVectorizer(max_features=128,stop_words=stop_words)
text_tfidf = sklearn_tfidf.fit_transform(df["tweet"])

In [147]:
y_final=np.array(df['target'])

In [148]:
# Modelling Helpers
from sklearn.model_selection import train_test_split
# Dividing the training set by using train_test_split
X_train, X_test, y_train, y_test = train_test_split(text_tfidf.toarray(), y_final, test_size=0.20, random_state=0)

In [149]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
#Logistic Regression with CountVectorizer
logreg = LogisticRegression(solver='lbfgs', max_iter=1000)
logreg.fit(X_train, y_train)
pred_logreg_tfidf = logreg.predict(X_test)
acc_logreg_tfidf = metrics.accuracy_score(y_test,pred_logreg_tfidf)
print(acc_logreg_tfidf)

0.8533834586466166


# Semantic 

In [138]:
# Modelling Helpers
from sklearn.model_selection import train_test_split
# Dividing the training set by using train_test_split
X_train, X_test, y_train, y_test = train_test_split(gl2ve, y_final, test_size=0.20, random_state=0)

In [139]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
#Logistic Regression with CountVectorizer
logreg = LogisticRegression(solver='lbfgs', max_iter=1000)
logreg.fit(X_train, y_train)
pred_logreg_tfidf = logreg.predict(X_test)
acc_logreg_tfidf = metrics.accuracy_score(y_test,pred_logreg_tfidf)
print(acc_logreg_tfidf)

0.7261009667024705


# Syntactic + Semantic

In [143]:
ssvec = np.concatenate((text_tfidf.toarray(), gl2ve), axis=1)#[text_tfidf[i].extend(g2ve[i]) for i in range(len(g2ve))]

In [144]:
# Modelling Helpers
from sklearn.model_selection import train_test_split
# Dividing the training set by using train_test_split
X_train, X_test, y_train, y_test = train_test_split(ssvec, y_final, test_size=0.20, random_state=0)

In [145]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
#Logistic Regression with CountVectorizer
logreg = LogisticRegression(solver='lbfgs', max_iter=1000)
logreg.fit(X_train, y_train)
pred_logreg_tfidf = logreg.predict(X_test)
acc_logreg_tfidf = metrics.accuracy_score(y_test,pred_logreg_tfidf)
print(acc_logreg_tfidf)

0.8576799140708915
